In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


* 导入mnist数据源
* 导入Sequential类，可以封装各种神经网络层，包括Dense全连接层，Dropout层，Cov2D 卷积层，等
* 导入使用的卷积层


# Sequential类---顺序模型


* 以通过将层的列表传递给 Sequential 的构造函数，来创建一个 Sequential 模型：



In [3]:
from keras.layers import Dense, Activation
model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

* 使用 add()方法添加

In [4]:
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation('relu'))

# 核心层

## dense层--普通的全连接层


keras.layers.Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)


Dense 实现以下操作： output = activation(dot(input, kernel) + bias) 其中  activation 是按逐个元素计算的激活函数，kernel 是由网络层创建的权值矩阵，以及  bias 是其创建的偏置向量 (只在 use_bias 为 True 时才有用)

* units: 正整数，输出空间维度。
* activation: 激活函数 (详见 activations)。 若不指定，则不使用激活函数 (即， “线性”激活:  a(x) = x)。
* use_bias: 布尔值，该层是否使用偏置向量。
* kernel_initializer: kernel 权值矩阵的初始化器 (详见 initializers)。
* bias_initializer: 偏置向量的初始化器 (see initializers).
* kernel_regularizer: 运用到 kernel 权值矩阵的正则化函数 (详见 regularizer)。
* bias_regularizer: 运用到偏置向的的正则化函数 (详见 regularizer)。
* activity_regularizer: 运用到层的输出的正则化函数 (它的 "activation")。 (详见 regularizer)。
* kernel_constraint: 运用到 kernel 权值矩阵的约束函数 (详见 constraints)。
* bias_constraint: 运用到偏置向量的约束函数 (详见 constraints)。

## Activation---激活函数


keras.layers.Activation(activation)

## Dropout--防止过拟合

* keras.layers.Dropout(rate, noise_shape=None, seed=None)
* rate: 在 0 和 1 之间浮动。需要丢弃的输入比例。


## Flatten---输入展平一维化

* keras.layers.Flatten(data_format=None)
* 参数表示展平的顺序

## Reshape---调整输入的尺寸


* keras.layers.Reshape(target_shape)



In [8]:
from keras.layers import Reshape
model = Sequential()
model.add(Reshape((3, 4), input_shape=(12,)))
# 现在：model.output_shape == (None, 3, 4)
# 注意： `None` 是批表示的维度

# 作为 Sequential 模型的中间层
model.add(Reshape((6, 2)))
# 现在： model.output_shape == (None, 6, 2)

# 还支持使用 `-1` 表示维度的尺寸推断
model.add(Reshape((-1, 2, 2)))
# 现在： model.output_shape == (None, 3, 2, 2)


# Conv2D--2D卷积层

* keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)


* filters: 整数，输出空间的维度 （即卷积中滤波器的输出数量）。

* kernel_size: 一个整数，或者 2 个整数表示的元组或列表， 指明 2D 卷积窗口的宽度和高度。 可以是一个整数，为所有空间维度指定相同的值

* strides: 一个整数，或者 2 个整数表示的元组或列表， 指明卷积沿宽度和高度方向的步长。 可以是一个整数，为所有空间维度指定相同的值。 指定任何 stride 值 != 1 与指定 dilation_rate 值 != 1 两者不兼容。



# MaxPooling2D---数据最大池化

* keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)

* pool_size: 整数，或者 2 个整数元组，（垂直方向，水平方向）缩小比例的因数。（2，2）会把输入张量的两个维度都缩小一半。 如果只使用一个整数，那么两个维度都会使用同样的窗口长度。
* strides: 整数，整数元组或者是 None。 步长值。 如果是 None，那么默认值是 pool_size。

# 准备训练集和测试集

In [9]:
# batch_size 太小会导致训练慢，过拟合等问题，太大会导致欠拟合。所以要适当选择
batch_size = 128
# 0-9手写数字一个有10个类别
num_classes = 10
# 12次完整迭代，差不多够了
epochs = 12
# 输入的图片是28*28像素的灰度图
img_rows, img_cols = 28, 28
# 训练集，测试集收集非常方便
(x_train, y_train), (x_test, y_test) = mnist.load_data()
 
# keras输入数据有两种格式，一种是通道数放在前面，一种是通道数放在后面，
# 其实就是格式差别而已
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
# 把数据变成float32更精确
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
# 把类别0-9变成2进制，方便训练
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# CNN构造


In [11]:
model = Sequential()
# 加上一个2D卷积层， 32个输出（也就是卷积通道），激活函数选用relu，
# 卷积核的窗口选用3*3像素窗口
model.add(Conv2D(32,(3,3),
                 activation='relu',
                 input_shape=input_shape,
                ))
# 64个通道的卷积层
model.add(Conv2D(64,(3,3), activation='relu',
                 ))
# 池化层是2*2像素的
model.add(MaxPooling2D(pool_size=(2, 2)))
# 对于池化层的输出，采用0.35概率的Dropout
model.add(Dropout(0.35))
# 展平所有像素，比如[28*28] -> [784]
model.add(Flatten())
# 对所有像素使用全连接层，输出为128，激活函数选用relu
model.add(Dense(128, activation='relu'))
# 对输入采用0.5概率的Dropout
model.add(Dropout(0.5))
# 对刚才Dropout的输出采用softmax激活函数，得到最后结果0-9
model.add(Dense(num_classes, activation='softmax'))



# CNN 训练


In [13]:
# 模型我们使用交叉熵损失函数，最优化方法选用Adadelta
model.compile(loss=keras.metrics.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
# 训练
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 104s 2ms/step - loss: 0.1707 - acc: 0.9495 - val_loss: 0.0551 - val_acc: 0.9827
Epoch 2/12
60000/60000 [==============================] - 103s 2ms/step - loss: 0.0914 - acc: 0.9723 - val_loss: 0.0404 - val_acc: 0.9862
Epoch 3/12
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0718 - acc: 0.9789 - val_loss: 0.0341 - val_acc: 0.9879
Epoch 4/12
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0585 - acc: 0.9827 - val_loss: 0.0364 - val_acc: 0.9873
Epoch 5/12
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0517 - acc: 0.9845 - val_loss: 0.0316 - val_acc: 0.9893
Epoch 6/12
60000/60000 [==============================] - 105s 2ms/step - loss: 0.0452 - acc: 0.9859 - val_loss: 0.0326 - val_acc: 0.9886
Epoch 7/12
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0422 - acc: 0.9874 - val_loss: 0.0271 - 

## 编译---compile（3个参数）

* 优化器 optimizer。它可以是现有优化器的字符串标识符，如 rmsprop 或 adagrad，也可以是 Optimizer 类的实例。详见：optimizers。

* 损失函数 loss，模型试图最小化的目标函数。它可以是现有损失函数的字符串标识符，如  categorical_crossentropy 或 mse，也可以是一个目标函数。详见：losses。

* 评估标准 metrics。对于任何分类问题，你都希望将其设置为 metrics = ['accuracy']。评估标准可以是现有的标准的字符串标识符，也可以是自定义的评估标准函数。


* 评估标准可以自定义

import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred])




# 训练--fit

> fit(self, x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

* x: 训练数据的 Numpy 数组。 如果模型中的输入层被命名，你也可以传递一个字典，将输入层名称映射到 Numpy 数组。 如果从本地框架张量馈送（例如 TensorFlow 数据张量）数据，x 可以是 None（默认）。

* y: 目标（标签）数据的 Numpy 数组。 如果模型中的输出层被命名，你也可以传递一个字典，将输出层名称映射到 Numpy 数组。 如果从本地框架张量馈送（例如 TensorFlow 数据张量）数据，y 可以是  None（默认）。

* batch_size: 整数或 None。每次提度更新的样本数。如果未指定，默认为 32.

* epochs: 整数。训练模型迭代轮次。一个轮次是在整个 x 或 y 上的一轮迭代。请注意，与  initial_epoch 一起，epochs 被理解为 「最终轮次」。模型并不是训练了 epochs 轮，而是到第  epochs 轮停止训练。

* verbose: 0, 1 或 2。日志显示模式。 0 = 安静模式, 1 = 进度条, 2 = 每轮一行。


* validation_data: 元组 (x_val，y_val) 或元组 (x_val，y_val，val_sample_weights)，用来评估损失，以及在每轮结束时的任何模型度量指标。模型将不会在这个数据上进行训练。这个参数会覆盖  validation_split。



# 总结

* 测试样本格式是28*28像素的1通道，灰度图，数量为60000个样本。

* 测试集是10000个样本。

* 一次epoch是一次完整迭代（所有样本都训练过），这里我们用了12次迭代，最后一次迭代就可以收敛到99.01%预测准确率了。

* loss是训练集损失值.  acc是训练集准确率。val_loss是测试集上的损失值，val_acc是测试集上的准确率。